# **Data-level Parallelism Integrating Project - MOVEMENT RECOGNITION IN SIMT**
Submitted by:
- Cai, Edison
- Dequico, Beverly Joyce
- La’O, Erin Denise
- Relucio, Jan Jhezaree

# Sequential

In [14]:
%%writefile group5_simt.c

#include <stdio.h>
#include <stdlib.h>
#include <ctype.h>
#include <math.h>
#include <string.h>
#include <stdint.h>
#include <time.h>

#define BILLION 1000000000L

// ------- R SQUARE -------
float calculateRSquare(int n, float *predict_y, float *actual_y) {
    float sum_x = 0, sum_y = 0, sum_xy = 0, sum_x2 = 0, sum_y2 = 0;

    // Compute the necessary sums
    for (int i = 0; i < n; i++) {
        sum_x += predict_y[i];
        sum_y += actual_y[i];
        sum_xy += predict_y[i] * actual_y[i];
        sum_x2 += predict_y[i] * predict_y[i];
        sum_y2 += actual_y[i] * actual_y[i];
    }

    // Compute the numerator and denominator of the R^2 formula
    float numerator = (n * sum_xy) - (sum_x * sum_y);
    float denominator = sqrt((n * sum_x2 - sum_x * sum_x) * (n * sum_y2 - sum_y * sum_y));

    if (denominator != 0) {
        return pow(numerator / denominator, 2);
    } else {
        return 0;  // Avoid division by zero
    }
}

// ------- GET COORDINATES -------
int checkCoordinates(char inputArr[], double x[], double y[], double z[]) {
    int open = 0;
    char current[10001];
    int index = 0;
    int count = 0;
    int doneX = 0;

    for (int i = 1; inputArr[i] != '\0'; i++) {
        if (inputArr[i] == '[') {
            open++;
        }

        else if ((inputArr[i] == '-') || (isdigit(inputArr[i]) || (inputArr[i] == '.'))) {
            current[index] = inputArr[i];
            index++;
        }

        else if ((inputArr[i] == ',') && (isdigit(inputArr[i+1]))) {
            if (doneX == 0) {
                x[count] = atof(current);
                doneX = 1;
            }
            else if (doneX == 1) {
                y[count] = atof(current);
            }

            //clear
            for (int w = 0; current[w] != '\0'; w++) {
                current[w] = '\0';
                index = 0;
            }
        }

        else if ((inputArr[i] == ']') && (inputArr[i+1] != ']')) {
            z[count] = atof(current);
            count++;
            doneX = 0;

            //clear
            for (int w = 0; current[w] != '\0'; w++) {
                current[w] = '\0';
                index = 0;
            }
        }
    }

    return (open);
}

// ------- ERROR CHECKING -------
int checkError(char inputArr[]) {
    int letterCount = 0;
    int open = 0;
    int close = 0;
    int errorCount = 0;

    // go through array
    for (int i = 0; inputArr[i] != '\0'; i++) {
        // check if array is an array of arrays
        if (inputArr[i+1] == '\0') {
            if ((inputArr[0] != '[') || (inputArr[1] != '[') || (inputArr[i-2] != ']') || (inputArr[i-1] != ']')) {
                printf("Input is not an array of arrays.\n");
                errorCount++;
            }
        }

      // check if there are letters in the input
        if ((inputArr[i] >= 'a' && inputArr[i] <= 'z') || (inputArr[i] >= 'A' && inputArr[i] <= 'Z')) {
            letterCount += 1; // invalid
        }

      // check for balanced brackets
      if (inputArr[i] == '[')
          open++;
      if (inputArr[i] == ']')
          close++;
    }

    if (letterCount > 0) {
        printf("No letters should be in the input.\n");
        errorCount++;
    }
    if (open != close) {
        printf("Brackets not balanced.\n");
        errorCount++;
    }

    if (errorCount > 0)
        return 0;
    if (errorCount == 0)
        return 1;
}

// ------- MAIN PROGRAM -------
int main() {
    char inputArr[10001];
    double x[10001];
    double y[10001];
    double z[10001];

    int inputIndex = 0;
    int sets = 0;
    int valid = 0;
    int type = 0;

    char row[10001];
    int count = 0;
    char* number;
    char name[500];

    while (valid == 0) {
        printf("Type of input:\n[1] .csv file\n[2] Manual Input\nINPUT: ");
        scanf("%d", &type);

        if ((type < 1) || (type > 2))
          printf("Invalid input entered. Try again.\n\n");
        else
          valid = 1;
    }

    valid = 0;

    if (type == 1) { // CSV FILE
      while (valid == 0) {
        printf("Enter file name (extension included): ");
        scanf("%s",name);

        for (int i=0; name[i] != '\0'; i++) {
          if (name[i+1] == '\0') {
            if ((name[i] != 'v') || (name[i-1] != 's') || (name[i-2] != 'c') || (name[i-3] != '.'))
              printf("File is not a .csv\n\n");
            else
              valid = 1;
          }
        }
      }

      FILE* fp = fopen(name,"r");

      if (!fp)
        printf("ERROR: File cannot be opened.\n");

      else {
        fgets(row, 10001, fp); //skip column names
        while (fgets(row, 10001, fp))
        {
          number = strtok(row, ",");
          x[count] = atof(number);
          number = strtok(NULL, ",");
          y[count] = atof(number);
          number = strtok(NULL, ",");
          z[count] = atof(number);
          //printf("%lf %lf %lf\n",x[count],y[count],z[count]);
          count++;
        }
      }

      sets = count;
    }

    else if (type == 2) { // MANUAL INPUT
        while (valid == 0) {
          printf("Input: ");
          fgets(inputArr, sizeof(inputArr), stdin);  // read string

        valid = checkError(inputArr);
        }

        // group arrays
        sets = checkCoordinates(inputArr,x,y,z);
    }

    valid = 0;

    while (valid == 0) {
      printf("\nEnter the index number: ");
      scanf("%d", &inputIndex);

      // check if inputSets <= number of sets in input array
      if ((inputIndex > sets) || (inputIndex <= 0))
          printf("Please input valid index. There are %d sets.\n",sets);
      else
          valid = 1;
    }

    float *actual_y, *predict_y, *r_square;

    for (int i = 0; i < sets; i++) {
        int index = inputIndex - 1;
        if (i == 0) {
            actual_y[i] = x[index];
            predict_y[i] = x[index+1];
            if (index+1 > sets)
                predict_y[i] = x[0];
        } else if (i == 1) {
            actual_y[i] = y[index];
            predict_y[i] = y[index+1];
            if (index+1 > sets)
                predict_y[i] = y[0];
        } else {
            actual_y[i] = z[index];
            predict_y[i] = z[index+1];
            if (index+1 > sets)
                predict_y[i] = z[0];
        }
    }


    // SOLUTION
    // timer reference: https://people.cs.rutgers.edu/~pxk/416/notes/c-tutorials/gettime.html
    uint64_t diff;
    struct timespec start, end;
    int i;

    // measure monotonic time
    clock_gettime(CLOCK_MONOTONIC, &start);	// mark start time

    // INSERT FUNCTION HERE
    calculateRSquare(sets, predict_y, actual_y); //!!!!!!!!!!!!

    clock_gettime(CLOCK_MONOTONIC, &end);	// mark the end time

    diff = BILLION * (end.tv_sec - start.tv_sec) + end.tv_nsec - start.tv_nsec;
    printf("elapsed time = %llu nanoseconds\n", (long long unsigned int) diff);

    // Results
    printf("\nRESULTS:\n");

    return 0;
}

Overwriting group5_simt.c


In [15]:
%%shell
gcc group5_simt.c -o group5_simt -lm -lrt
./group5_simt

Type of input:
[1] .csv file
[2] Manual Input
INPUT: 1
Enter file name (extension included): xyz2.csv

Enter the index number: 2


CalledProcessError: Command 'gcc group5_simt.c -o group5_simt -lm -lrt
./group5_simt
' died with <Signals.SIGSEGV: 11>.

# **CUDA - No Prefetching**

In [9]:
%%writefile CUDA_movement.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__
void rSquare(size_t n, float *r_square, float *predict_y, float *actual_y) {
    extern __shared__ float shared_mem[];
    float *sum_x = &shared_mem[0];
    float *sum_y = &shared_mem[1];
    float *sum_xy = &shared_mem[2];
    float *sum_x2 = &shared_mem[3];
    float *sum_y2 = &shared_mem[4];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    // INITIALIZE SHARED MEMORY VALUES
    if (threadIdx.x == 0) {
        *sum_x = 0;
        *sum_y = 0;
        *sum_xy = 0;
        *sum_x2 = 0;
        *sum_y2 = 0;
    }

    __syncthreads();

    // CALCULATE SUMS IN PARALLEL
    for (int i = index; i < n; i += stride) {
        atomicAdd(sum_x, predict_y[i]);
        atomicAdd(sum_y, actual_y[i]);
        atomicAdd(sum_xy, predict_y[i] * actual_y[i]);
        atomicAdd(sum_x2, predict_y[i] * predict_y[i]);
        atomicAdd(sum_y2, actual_y[i] * actual_y[i]);
    }

    __syncthreads();

    // COMPUTE THE RESULT IN THE FIRST THREAD OF EACH BLOCK
    if (threadIdx.x == 0) {
        float numerator = (n * (*sum_xy)) - (*sum_x * *sum_y);
        float denominator = sqrtf((n * (*sum_x2) - (*sum_x) * (*sum_x)) *
                                   (n * (*sum_y2) - (*sum_y) * (*sum_y)));

        if (denominator != 0) {
            *r_square = powf(numerator / denominator, 2);
        } else {
            *r_square = 0;
        }
    }
}

int main() {
    //const size_t ARRAY_SIZE = 60;
    size_t ARRAY_SIZE;
    // const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 1;

    double x[10001];
    double y[10001];
    double z[10001];

    char row[10001];
    int count = 0;
    char* number;
    char name[500];
    char* temp;

    printf("Enter file name (extension included): ");
    scanf("%s",name);

    FILE* fp = fopen(name,"r");

    if (!fp)
      printf("ERROR: File cannot be opened.\n");

    else {
      fgets(row, 10001, fp); //skip column names
      while (fgets(row, 10001, fp))
      {
        temp=row;
        number = strtok_r(row, ",", &temp);
        x[count] = atof(number);
        number = strtok_r(NULL, ",", &temp);
        y[count] = atof(number);
        number = strtok_r(NULL, ",", &temp);
        z[count] = atof(number);
        //printf("%lf %lf %lf\n",x[count],y[count],z[count]);
        count++;
      }
      ARRAY_SIZE = count;

      int group_index;
      printf("Enter the index number: ");
      scanf("%d", &group_index);

      const size_t num_elements = 3;

      float *actual_y, *predict_y, *r_square;
      cudaMallocManaged(&actual_y, num_elements * sizeof(float));
      cudaMallocManaged(&predict_y, num_elements * sizeof(float));
      cudaMallocManaged(&r_square, sizeof(float));

      for (int i = 0; i < num_elements; i++) {
          int index = group_index - 1;
          if (i == 0) {
              actual_y[i] = x[index];
              predict_y[i] = x[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = x[0];
          } else if (i == 1) {
              actual_y[i] = y[index];
              predict_y[i] = y[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = y[0];
          } else {
              actual_y[i] = z[index];
              predict_y[i] = z[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = z[0];
          }
      }

      size_t numThreads = 1024;
      size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

      printf("*** rSquare function ***\n");
      printf("numElements = %lu\n", ARRAY_SIZE);
      printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

      *r_square = 0;
      for (size_t i = 0; i < loope; i++) {
          // SHARED MEMORY SIZE IS 5 * SIZEOF(FLOAT) FOR THE FIVE VARIABLES
          rSquare<<<numBlocks, numThreads, 5 * sizeof(float)>>>(num_elements, r_square, predict_y, actual_y);
          cudaDeviceSynchronize();
      }

      printf("R^2: %f\n", *r_square);

      if (*r_square >= 0.9)
        printf("Movement is linear!");

      cudaFree(actual_y);
      cudaFree(predict_y);
      cudaFree(r_square);
    }

    return 0;
}


Overwriting CUDA_movement.cu


In [10]:
%%shell
nvcc CUDA_movement.cu -o CUDA_movement
nvprof ./CUDA_movement

Enter file name (extension included): xyz2.csv
Enter the index number: 2
==1837== NVPROF is profiling process 1837, command: ./CUDA_movement
*** rSquare function ***
numElements = 7681
numBlocks = 8, numThreads = 1024 
R^2: 0.992948
Movement is linear!==1837== Profiling application: ./CUDA_movement
==1837== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  390.17us         1  390.17us  390.17us  390.17us  rSquare(unsigned long, float*, float*, float*)
      API calls:   99.10%  104.70ms         3  34.899ms  9.8320us  104.67ms  cudaMallocManaged
                    0.38%  397.48us         1  397.48us  397.48us  397.48us  cudaDeviceSynchronize
                    0.23%  239.23us         1  239.23us  239.23us  239.23us  cudaLaunchKernel
                    0.15%  155.24us         3  51.747us  10.474us  116.41us  cudaFree
                    0.13%  132.40us       114  1.1610us     144ns  52.189us  cuDeviceGetAttri

# **CUDA - Prefetch Memory**

In [7]:
%%writefile CUDA_movement.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__
void rSquare(size_t n, float *r_square, float *predict_y, float *actual_y) {
    extern __shared__ float shared_mem[];
    float *sum_x = &shared_mem[0];
    float *sum_y = &shared_mem[1];
    float *sum_xy = &shared_mem[2];
    float *sum_x2 = &shared_mem[3];
    float *sum_y2 = &shared_mem[4];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    // INITIALIZE SHARED MEMORY VALUES
    if (threadIdx.x == 0) {
        *sum_x = 0;
        *sum_y = 0;
        *sum_xy = 0;
        *sum_x2 = 0;
        *sum_y2 = 0;
    }

    __syncthreads();

    // CALCULATE SUMS IN PARALLEL
    for (int i = index; i < n; i += stride) {
        atomicAdd(sum_x, predict_y[i]);
        atomicAdd(sum_y, actual_y[i]);
        atomicAdd(sum_xy, predict_y[i] * actual_y[i]);
        atomicAdd(sum_x2, predict_y[i] * predict_y[i]);
        atomicAdd(sum_y2, actual_y[i] * actual_y[i]);
    }

    __syncthreads();

    // COMPUTE THE RESULT IN THE FIRST THREAD OF EACH BLOCK
    if (threadIdx.x == 0) {
        float numerator = (n * (*sum_xy)) - (*sum_x * *sum_y);
        float denominator = sqrtf((n * (*sum_x2) - (*sum_x) * (*sum_x)) *
                                   (n * (*sum_y2) - (*sum_y) * (*sum_y)));

        if (denominator != 0) {
            *r_square = powf(numerator / denominator, 2);
        } else {
            *r_square = 0;
        }
    }
}

int main() {
    const size_t ARRAY_SIZE = 60;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const size_t loope = 1;

    double x[10001];
    double y[10001];
    double z[10001];
    char row[10001];
    int count = 0;
    char* number;
    char name[500];
    char* temp;

    printf("Enter file name (extension included): ");
    scanf("%s",name);

    FILE* fp = fopen(name,"r");

    if (!fp)
      printf("ERROR: File cannot be opened.\n");

    else {
      fgets(row, 10001, fp); //skip column names
      while (fgets(row, 10001, fp))
      {
        temp=row;
        number = strtok_r(row, ",", &temp);
        x[count] = atof(number);
        number = strtok_r(NULL, ",", &temp);
        y[count] = atof(number);
        number = strtok_r(NULL, ",", &temp);
        z[count] = atof(number);
        //printf("%lf %lf %lf\n",x[count],y[count],z[count]);
        count++;
      }

      int group_index;
      printf("Enter the index number: ");
      scanf("%d", &group_index);

      const size_t num_elements = 3;

      float *actual_y, *predict_y, *r_square;
      cudaMallocManaged(&actual_y, num_elements * sizeof(float));
      cudaMallocManaged(&predict_y, num_elements * sizeof(float));
      cudaMallocManaged(&r_square, sizeof(float));

      int device;
      cudaGetDevice(&device);
      cudaMemAdvise(actual_y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
      cudaMemAdvise(predict_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
      cudaMemPrefetchAsync(r_square, ARRAY_BYTES, device, NULL);

      for (int i = 0; i < num_elements; i++) {
          int index = group_index - 1;
          if (i == 0) {
              actual_y[i] = x[index];
              predict_y[i] = x[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = x[0];
          } else if (i == 1) {
              actual_y[i] = y[index];
              predict_y[i] = y[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = y[0];
          } else {
              actual_y[i] = z[index];
              predict_y[i] = z[index+1];
              if (index+1 > ARRAY_SIZE)
                  predict_y[i] = z[0];
          }
      }

      cudaMemPrefetchAsync(actual_y, ARRAY_BYTES, device, NULL);
      cudaMemPrefetchAsync(predict_y, ARRAY_BYTES, device, NULL);

      size_t numThreads = 1024;
      size_t numBlocks = (ARRAY_SIZE + numThreads - 1) / numThreads;

      printf("*** rSquare function ***\n");
      printf("numElements = %lu\n", ARRAY_SIZE);
      printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

      *r_square = 0;
      for (size_t i = 0; i < loope; i++) {
          // SHARED MEMORY SIZE IS 5 * SIZEOF(FLOAT) FOR THE FIVE VARIABLES
          rSquare<<<numBlocks, numThreads, 5 * sizeof(float)>>>(num_elements, r_square, predict_y, actual_y);
          cudaDeviceSynchronize();
      }

      printf("R^2: %f\n", *r_square);

      if (*r_square >= 0.9)
        printf("Movement is linear!");

      cudaFree(actual_y);
      cudaFree(predict_y);
      cudaFree(r_square);
    }

    return 0;
}


Overwriting CUDA_movement.cu


In [8]:
%%shell
nvcc CUDA_movement.cu -o CUDA_movement
nvprof ./CUDA_movement

Enter file name (extension included): xyz2.csv
Enter the index number: 2
==1543== NVPROF is profiling process 1543, command: ./CUDA_movement
*** rSquare function ***
numElements = 60
numBlocks = 1, numThreads = 1024 
R^2: 0.992948
Movement is linear!==1543== Profiling application: ./CUDA_movement
==1543== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  427.61us         1  427.61us  427.61us  427.61us  rSquare(unsigned long, float*, float*, float*)
      API calls:   51.33%  140.71ms         3  46.903ms  5.5190us  140.67ms  cudaMallocManaged
                   47.96%  131.47ms         1  131.47ms  131.47ms  131.47ms  cudaLaunchKernel
                    0.41%  1.1308ms         3  376.92us  1.1090us  1.1261ms  cudaMemPrefetchAsync
                    0.16%  431.35us         1  431.35us  431.35us  431.35us  cudaDeviceSynchronize
                    0.08%  230.20us         3  76.734us  12.572us  166.06us  cudaFr